In [ ]:
import sys
import os
# third party
import numpy as np
import pandas as pd

package_path = '../CTAB-GAN-main'

sys.path.append(package_path)

from model.ctabgan import CTABGAN
# Importing the evaluation metrics 
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
# Importing standard libraries
import numpy as np
import pandas as pd
import glob


# Specifying the list of classifiers to conduct ML utility evaluation
real_path = "../CTAB-GAN-main/Real_Datasets/Adult3.csv"
#real_path = '../CTGAN-main/CTGAN-main/examples/csv/train_clean.csv'
#real_path = "../synthcity-main/tutorials/covertype_preprocessed.csv"
#fake_paths = ["../synthcity-main/tutorials/DDPM-cover-5.csv"]

#real_path = r"C:\Users\26332\Desktop\Churn_Modelling_processed.csv"
#fake_paths = [r"..\synthcity-main\tutorials\duanwen-trans-adu.csv"]
# real_path = "G:/DataSets/Adult.csv"
# real_path = 'G:/DataSets/adult_processed_0.csv'
fake_paths = ["G:/DataSets/Fake_Dataset/TransCTGAN-finaladu200_5.csv"]
# fake_paths = ["G:/DataSets/Fake_Dataset/TransCTGAN-200eAdult3.csv"]
# real_path = "../CTAB-GAN-main/Real_Datasets/Adult3.csv"
#fake_paths = ["../fakeData_CTGAN-Adult.csv"]
classifiers_list = ["lr","dt","rf","mlp","svm"]
# Storing and presenting the results as a dataframe
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)
result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

2
2
2
2
2
2
2
2
2
2


,Acc,AUC,F1_Score
lr,2.75,0.032865,0.000801
dt,5.90,0.069439,0.074687
rf,3.85,0.045534,0.075389
mlp,3.00,0.035053,-0.022103
svm,4.00,0.044560,-0.075337


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from scipy.spatial.distance import cdist

def kl_divergence(p, q):

    epsilon = 1e-10
    p = np.clip(p, epsilon, 1)
    q = np.clip(q, epsilon, 1)
    return np.sum(p * np.log(p / q))

def js_divergence(p, q):

    m = 0.5 * (p + q)
    return 0.5 * (kl_divergence(p, m) + kl_divergence(q, m))

def wasserstein_distance(p, q):

    return np.sum(np.abs(np.cumsum(p) - np.cumsum(q)))

def handle_missing_data(data):

    return data.dropna()

def handle_zero_variance_columns(data):

    return data.loc[:, data.var() > 0]

def calculate_metrics(real_data, generated_data):

    real_data = handle_missing_data(real_data)
    generated_data = handle_missing_data(generated_data)

    real_data = handle_zero_variance_columns(real_data)
    generated_data = handle_zero_variance_columns(generated_data)

    real_data = (real_data - real_data.mean(axis=0)) / real_data.std(axis=0)
    generated_data = (generated_data - generated_data.mean(axis=0)) / generated_data.std(axis=0)

    if real_data.isnull().any().any() or generated_data.isnull().any().any():
        print("Warning: There are still NaN values after handling!")
        return None, None, None
    
    real_hist = np.histogram(real_data, bins=30, density=True)[0]
    generated_hist = np.histogram(generated_data, bins=30, density=True)[0]

    kl = kl_divergence(real_hist, generated_hist)
    
    jsd = js_divergence(real_hist, generated_hist)

    wasserstein = wasserstein_distance(real_hist, generated_hist)

    return kl, jsd, wasserstein

def load_data(path):
    return pd.read_csv(path)
#real_path = '../CTGAN-main/CTGAN-main/examples/csv/train_clean.csv'
#real_path = "../CTAB-GAN-main/Real_Datasets/Adult3.csv"
#real_path = "../CTAB-GAN-main/Real_Datasets/CreditLong2.csv"
#real_path = "../synthcity-main/tutorials/covertype_preprocessed.csv"
#fake_path = "../synthcity-main/tutorials/ETransCTGAN-Cov_5.csv"
#fake_path = "../synthcity-main/tutorials/newtrans_calc_pro_cov.csv"

#fake_path = r"..\synthcity-main\tutorials\duanwen-trans-cre.csv"
# fake_path = "G:/DataSets/Fake_Dataset/CTABGAN+-Churn300e2.csv"
#real_path = "G:/DataSets/Churn.csv"
# real_path = "G:/DataSets/Adult.csv"
# fake_path = "G:/DataSets/Fake_Dataset/CTGAN-adu100e.csv"
# fake_path = 'G:/DataSets/Fake_Dataset/CTABGAN-Credit150k0.csv'
real_path = 'G:/DataSets/Credit150k.csv'
# fake_path = "G:/DataSets/Fake_Dataset/TransCTGAN-200eAdult3.csv"
#real_path = "../CTAB-GAN-main/Real_Datasets/Adult3.csv"
# fake_path = "G:/DataSets/Fake_Dataset/TVAE-200eCover.csv"
# real_path = 'G:\DataSets\Covertype30k.csv'
# real_path = '../CTGAN-main/CTGAN-main/examples/csv/train_clean.csv'
# fake_path = 'G:/DataSets/Fake_Dataset/Synthcity-transctgan-tit300.csv'
#real_path = 'G:\DataSets\Covertype30k.csv'
fake_path = 'G:/DataSets/Fake_Dataset/zhexian-realctgan-credit150k-25e.csv'
# real_path = "G:/DataSets/Churn.csv"
real_data = load_data(real_path)
generated_data = load_data(fake_path)

kl, jsd, wasserstein = calculate_metrics(real_data, generated_data)

if kl is not None and jsd is not None and wasserstein is not None:
    print(f"KL Divergence: {kl}")
    print(f"Jensen-Shannon Divergence: {jsd}")
    print(f"Wasserstein Distance: {wasserstein}")
else:
    print("can't be calculated")

KL Divergence: -0.27343873333828594
Jensen-Shannon Divergence: 0.13372765812366655
Wasserstein Distance: 10.007742818711975


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.decomposition import PCA
# from scipy.stats import entropy
# from scipy.spatial.distance import cdist
# import scipy

# def compute_jsd(real_data, generated_data, bins=30, pca_dim=10):
#     """
#     Compute Jensen-Shannon Divergence (JSD) between real and generated data with PCA dimensionality reduction.

#     Parameters:
#     - real_data: Real dataset (numpy array)
#     - generated_data: Generated dataset (numpy array)
#     - bins: Number of bins for histogram (int)
#     - pca_dim: Number of dimensions for PCA (int)
#     """
#     assert real_data.shape == generated_data.shape, "Real and generated data must have the same shape."
    
#     # Ensure data is in the right format
#     real_data = np.nan_to_num(real_data)
#     generated_data = np.nan_to_num(generated_data)

#     # Apply PCA for dimensionality reduction
#     pca = PCA(n_components=pca_dim)
#     real_data_pca = pca.fit_transform(real_data)
#     generated_data_pca = pca.transform(generated_data)

#     # Dynamic range calculation
#     min_vals = np.minimum(real_data_pca.min(axis=0), generated_data_pca.min(axis=0))
#     max_vals = np.maximum(real_data_pca.max(axis=0), generated_data_pca.max(axis=0))
#     range_vals = [(min_val, max_val) for min_val, max_val in zip(min_vals, max_vals)]

#     # Compute histograms
#     real_hist, _ = np.histogramdd(real_data_pca, bins=bins, range=range_vals, density=True)
#     generated_hist, _ = np.histogramdd(generated_data_pca, bins=bins, range=range_vals, density=True)

#     # Flatten and normalize
#     real_hist_flat = real_hist.flatten() + 1e-8
#     generated_hist_flat = generated_hist.flatten() + 1e-8
#     real_hist_flat /= real_hist_flat.sum()
#     generated_hist_flat /= generated_hist_flat.sum()

#     # Compute JSD
#     m = 0.5 * (real_hist_flat + generated_hist_flat)
#     jsd = 0.5 * (entropy(real_hist_flat, m) + entropy(generated_hist_flat, m))
#     return jsd

# def compute_kl(real_data, generated_data, bins=30, pca_dim=10):
#     """
#     Compute Kullback-Leibler Divergence (KL) between real and generated data with PCA dimensionality reduction.

#     Parameters:
#     - real_data: Real dataset (numpy array)
#     - generated_data: Generated dataset (numpy array)
#     - bins: Number of bins for histogram (int)
#     - pca_dim: Number of dimensions for PCA (int)
#     """
#     assert real_data.shape == generated_data.shape, "Real and generated data must have the same shape."
    
#     # Ensure data is in the right format
#     real_data = np.nan_to_num(real_data)
#     generated_data = np.nan_to_num(generated_data)

#     # Apply PCA for dimensionality reduction
#     pca = PCA(n_components=pca_dim)
#     real_data_pca = pca.fit_transform(real_data)
#     generated_data_pca = pca.transform(generated_data)

#     # Dynamic range calculation
#     min_vals = np.minimum(real_data_pca.min(axis=0), generated_data_pca.min(axis=0))
#     max_vals = np.maximum(real_data_pca.max(axis=0), generated_data_pca.max(axis=0))
#     range_vals = [(min_val, max_val) for min_val, max_val in zip(min_vals, max_vals)]

#     # Compute histograms
#     real_hist, _ = np.histogramdd(real_data_pca, bins=bins, range=range_vals, density=True)
#     generated_hist, _ = np.histogramdd(generated_data_pca, bins=bins, range=range_vals, density=True)

#     # Flatten and normalize
#     real_hist_flat = real_hist.flatten() + 1e-8
#     generated_hist_flat = generated_hist.flatten() + 1e-8
#     real_hist_flat /= real_hist_flat.sum()
#     generated_hist_flat /= generated_hist_flat.sum()

#     # Compute KL
#     kl = entropy(real_hist_flat, generated_hist_flat)
#     return kl

# def compute_wd(real_data, generated_data, pca_dim=None):
#     """
#     Compute Wasserstein Distance (WD) between real and generated data with optional PCA.

#     Parameters:
#     - real_data: Real dataset (numpy array)
#     - generated_data: Generated dataset (numpy array)
#     - pca_dim: Number of dimensions for PCA (int, optional)
#     """
#     assert real_data.shape == generated_data.shape, "Real and generated data must have the same shape."
    
#     # Ensure data is in the right format
#     real_data = np.nan_to_num(real_data)
#     generated_data = np.nan_to_num(generated_data)

#     if pca_dim is not None and pca_dim < real_data.shape[1]:
#         # Apply PCA for dimensionality reduction
#         pca = PCA(n_components=pca_dim)
#         real_data = pca.fit_transform(real_data)
#         generated_data = pca.transform(generated_data)
    
#     # Compute WD
#     wd = cdist(real_data, generated_data, metric='euclidean').mean()
#     return wd

# def evaluate_datasets(real_path, fake_path, pca_dim_js_kl=10, pca_dim_wd=None):
#     """
#     Evaluate generated data against real data using WD, JSD, and KL.

#     Parameters:
#     - real_path: Path to real dataset (CSV format)
#     - fake_path: Path to generated dataset (CSV format)
#     - pca_dim_js_kl: Number of dimensions for PCA in JSD and KL computation (int)
#     - pca_dim_wd: Number of dimensions for PCA in WD computation (int, optional)
#     """
#     # Load datasets
#     real_data = pd.read_csv(real_path).values
#     generated_data = pd.read_csv(fake_path).values

#     # Ensure data has the same shape
#     assert real_data.shape[1] == generated_data.shape[1], "Real and generated data must have the same number of features."
#     min_samples = min(real_data.shape[0], generated_data.shape[0])
#     real_data = real_data[:min_samples]
#     generated_data = generated_data[:min_samples]

#     # Compute metrics
#     wd = compute_wd(real_data, generated_data, pca_dim_wd)
#     jsd = compute_jsd(real_data, generated_data, pca_dim=pca_dim_js_kl)
#     kl = compute_kl(real_data, generated_data, pca_dim=pca_dim_js_kl)

#     return {
#         "Wasserstein Distance": wd,
#         "Jensen-Shannon Divergence": jsd,
#         "Kullback-Leibler Divergence": kl
#     }